In [2]:
import datetime
import calendar

def timestamp_to_epcoch_day(timestamp):
    if type(timestamp) is str:
        timestamp = datetime.datetime.strptime(timestamp[0:-5], '%Y-%m-%dT%H:%M:%S')
    return calendar.timegm(timestamp.timetuple()) % (3600 * 24)


# Provide current timestamp
timestamp = datetime.datetime.now()
timestamp_to_epcoch_day(timestamp)

61432

In [3]:
data_sample.timestamp.apply(timestamp_to_epcoch_day)

NameError: name 'data_sample' is not defined

In [4]:
data_sample.event.value_counts()

NameError: name 'data_sample' is not defined

In [2]:
def getTimeFromTimestamp(timestamp):
    sum = 0
    sum += int(timestamp[:2]) * 3600
    sum += int(timestamp[3:5]) * 60
    sum += int(timestamp[6:8])
    return sum

def getTimestampFromTime(time):
    hour = time // 3600
    minute = (time - hour * 3600) // 60
    second = time - hour * 3600 - minute * 60
    return str(hour).zfill(2) + ':' + str(minute).zfill(2) + ':' + str(second).zfill(2)

In [3]:
def getStartStep(time):
    time415 = getTimeFromTimestamp("04:15:00")  
    timeStart = getTimeFromTimestamp(time[11:-5])

    if timeStart < time415:
        return (time415 - timeStart) // 5
    else:
        return ((getTimeFromTimestamp("24:00:00") - timeStart) + time415) // 5

In [13]:
#creats a list of shape(batch_size, context_length, 2)

stepsPerNight = 24 * 3600 // 5
contextLength = getTimeFromTimestamp("08:00:00") // 5
array = []
labelsArray = []

for person in data['series_id'].unique():
    data_person = data[data['series_id'] == person]
    data_person = data_person.reset_index()

    label_person = labels[labels['series_id'] == person]
    label_person = label_person.reset_index()


    startStep = getStartStep(data_person["timestamp"][0])
    numDays = len(data_person) // stepsPerNight


    for day in range(numDays - 1):
        nightLabel = label_person[(label_person['step'] > startStep + (day * stepsPerNight)) & (label_person['step'] < startStep + contextLength + (day * stepsPerNight))]
        dayData = data[startStep + (day * stepsPerNight) : startStep + contextLength + (day * stepsPerNight)]

        array.append([dayData["enmo"], dayData["anglez"]])

        try:
            labelsArray.append(float(nightLabel["step"] - (startStep + stepsPerNight * day)))
        except:
            labelsArray.append(5759.0)


    #only for debugging
    #break
    


1    1752.0
Name: step, dtype: float64
pass
3    1032.0
Name: step, dtype: float64
pass
5    660.0
Name: step, dtype: float64
pass
7    1836.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
11    2280.0
Name: step, dtype: float64
pass
13    5664.0
Name: step, dtype: float64
pass
15    3192.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
19    2700.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
23    3012.0
Name: step, dtype: float64
pass
25    2844.0
Name: step, dtype: float64
pass
Series([], Name: step, dtype: float64)
29    2604.0
Name: step, dtype: float64
pass
31    1752.0
Name: step, dtype: float64
pass
33    2112.0
Name: step, dtype: float64
pass
35    2292.0
Name: step, dtype: float64
pass
37    2376.0
Name: step, dtype: float64
pass
39    1560.0
Name: step, dtype: float64
pass
41    1164.0
Name: step, dtype: float64
pass


In [11]:
import numpy as np

numpyData = np.array(array)
numpyLabels = np.array(labelsArray)

np.save("batched_data\\features", numpyData)
np.save("batched_data\\labels", numpyLabels)
